In [2]:
'''


We want to test if the qiskit model, when inserted the Lambeq paramters, returns the correct labels most of the times. 
'''
from lambeq import BobcatParser
parser = BobcatParser(verbose='text')
from lambeq import RemoveCupsRewriter
remove_cups = RemoveCupsRewriter()

from lambeq import AtomicType, IQPAnsatz
ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1},
                   n_layers=1, n_single_qubit_params=3)





# load dictionary with pickle
import pickle 


gijs = False

if gijs == True: 
    # Numpymodel 
    with open('../weights/weightsDictNP1QB_gijs_500.pkl', 'rb') as fp: 
        parameterDictNP = pickle.load(fp)

    # Numpymodel 
    with open('../weights/weightsDictTK1QB_gijs_500.pkl', 'rb') as fp: 
        parameterDictTK = pickle.load(fp)

    # Pennylane model 
    with open('../weights/weightsDictPL1QB_gijs_500.pkl', 'rb') as fp: 
        parameterDictPL = pickle.load(fp)

else: 
    # Numpymodel 
    with open('../weights/weightsDictNP1QB.pkl', 'rb') as fp: 
        parameterDictNP = pickle.load(fp)

    # Numpymodel 
    with open('../weights/weightsDictTK1QB.pkl', 'rb') as fp: 
        parameterDictTK = pickle.load(fp)

    # Pennylane model 
    with open('../weights/weightsDictPL1QB.pkl', 'rb') as fp: 
        parameterDictPL = pickle.load(fp)


/home/bob/anaconda3/envs/code/lib/python3.9/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [13]:

from pytket.extensions.qiskit import tk_to_qiskit
import qiskit.quantum_info as qi

parameterDict = parameterDictTK

def ReturnLabel(sentence: str):
    '''
    
    '''
    diagram = parser.sentence2diagram(sentence= sentence, suppress_exceptions=True)

    diagram = remove_cups(diagram)

    circuit = ansatz(diagram= diagram)

    tempCircTK = circuit.to_tk()

    tempCircQiskit = tk_to_qiskit(tempCircTK)

    print(tempCircQiskit)

    alteredTempCirc, measuredQubits = SetUpQiskitDiagram(tempCircQiskit, parameterDict)

    print(alteredTempCirc)


    # prepare state to 000...
    stateVector = qi.Statevector.from_label('000')
    # evolve state
    stateVector = stateVector.evolve(alteredTempCirc)

    print('measured qubits: ', measuredQubits)

    stateVector.measure(measuredQubits)
    
    dM = qi.partial_trace(stateVector, [0, 2])

    print('the density matrix is: ', dM)

    # define matrices for food and IT to compare to the statevector
    iTMatrix = np.array([[0,0], [0,1]])
    foodMatrix = np.array([[1,0], [0,0]])

    iTDM = qi.DensityMatrix(iTMatrix)
    foodDM = qi.DensityMatrix(foodMatrix)

    if qi.state_fidelity(iTDM, dM) < qi.state_fidelity(foodDM, dM):
        return 'food'
    else: 
        return 'IT'



label = ReturnLabel('man cooks sauce')

print('final label is: ', label)


     ┌───┐                                          ┌───┐             »
q_0: ┤ H ├──────────────■───────────────────────────┤ H ├─────────────»
     ├───┤┌─────────────┴──────────────┐            └───┘             »
q_1: ┤ H ├┤ Rz(2*π*cooks__n.r@s@n.l_0) ├──────────────■───────────────»
     ├───┤└────────────────────────────┘┌─────────────┴──────────────┐»
q_2: ┤ H ├──────────────────────────────┤ Rz(2*π*cooks__n.r@s@n.l_1) ├»
     └───┘                              └────────────────────────────┘»
c: 2/═════════════════════════════════════════════════════════════════»
                                                                      »
«     ┌────────────────────┐ ┌────────────────────┐  ┌────────────────────┐ ┌─┐»
«q_0: ┤ Rx(-2*π*man†_n__0) ├─┤ Rz(-2*π*man†_n__1) ├──┤ Rx(-2*π*man†_n__2) ├─┤M├»
«     └───────┬───┬────────┘ └────────────────────┘  └────────────────────┘ └╥┘»
«q_1: ────────┤ H ├──────────────────────────────────────────────────────────╫─»
«             ├───┤         

/tmp/ipykernel_15443/3881456281.py:57: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  measuredQubits.append(circuitData[i].qubits[0].index)


In [4]:
import numpy as np
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.converters import dag_to_circuit, circuit_to_dag

def SetUpQiskitDiagram(circuit: QuantumCircuit, parameterDict: dict, show: bool = False):
    '''
    
    '''
    # start by extracting circuit data 
    circuitData = circuit.data

    if show:     
        print('circuit that we start with: ')
        print(circuit)

    # FIRST: UPDATE THE PARAMETERS, THEN REMOVE THE GATES FROM THE CIRCUIT 

    # get parameters of the remaining circuit 
    params = circuit.parameters

    # set the parameters of the remaining words based on the models weights 
    for i in range(len(params)):
        # print(params[i])
        if str(params[i]) in parameterDict:
            circuit = circuit.assign_parameters({params[i]: (parameterDict[str(params[i])])})#/(2*np.pi))})
        else: 
            print('TESTSETESTSETS')
            # if the model only learned parameters for the word that are not daggered
            if '†' in str(params[i]):
                tempParam = str(params[i])
                # reverse string 
                tempParam = tempParam[::-1]

                # remove elements 2,3,4,5
                tempParam = tempParam[:2] + 'n__' + tempParam[5:]

                tempParam = tempParam[::-1]
                print(tempParam)             

                circuit = circuit.assign_parameters({params[i]: -(parameterDict[tempParam])})#/(2*np.pi))}) 


    if show: 
        print('circuit with new params: ')
        print(circuit)

    measuredQubits = []

    i = len(circuitData) - 1

    # remove measurements 
    while i >= 0: 
        if str(circuitData[i].operation.name) == 'measure': 
            print('removed measurement. ')

            measuredQubits.append(circuitData[i].qubits[0].index)

            del circuit.data[i]
        i -= 1

    if show:
        print('removeed measurements?')
        print(circuit)

    return circuit, measuredQubits
